# Gemini API 實作

### 0. 前言

前面講了這麼多，其實是在跟大家講解API的實際工作原理。<br/>
Google generative AI 的套件已經將整個APIc calling 的操作整合，大家不用擔心操作複雜。<br/>
那麼接下來我們就來實作一個簡單的API calling

### 1. 預先下載套件

In [1]:
%pip install google-generativeai

Note: you may need to restart the kernel to use updated packages.


### 2. Config API 金鑰及指定模型

In [2]:
import google.generativeai as genai
import os

genai.configure(api_key="YOUR_API_KEY")
model = genai.GenerativeModel("gemini-1.5-flash")


d:\miniconda\envs\mcsh_course\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 3. Generate content 生成文字

In [3]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("What can i do with ai?")
print(response)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "AI has a vast range of applications, impacting nearly every aspect of modern life.  Here are some examples categorized for clarity:\n\n**Creative & Entertainment:**\n\n* **Generating text:** Write stories, poems, scripts, articles, marketing copy, and more. Tools like ChatGPT and Jasper are examples.\n* **Creating images:** Generate unique artwork, illustrations, logos, and photorealistic images from text descriptions using tools like Midjourney, DALL-E 2, and Stable Diffusion.\n* **Composing music:** Create original musical pieces in various styles.\n* **Editing video and audio:** Enhance video quality, add special effects, and improve audio clarity.  AI can also assist in creating realistic deepfakes (though ethically questionable in many applications).\n*

查看重點回覆

In [4]:
print(response.text)

AI has a vast range of applications, impacting nearly every aspect of modern life.  Here are some examples categorized for clarity:

**Creative & Entertainment:**

* **Generating text:** Write stories, poems, scripts, articles, marketing copy, and more. Tools like ChatGPT and Jasper are examples.
* **Creating images:** Generate unique artwork, illustrations, logos, and photorealistic images from text descriptions using tools like Midjourney, DALL-E 2, and Stable Diffusion.
* **Composing music:** Create original musical pieces in various styles.
* **Editing video and audio:** Enhance video quality, add special effects, and improve audio clarity.  AI can also assist in creating realistic deepfakes (though ethically questionable in many applications).
* **Playing games:** AI powers opponents in video games, offering increasingly challenging and realistic gameplay.


**Productivity & Business:**

* **Automating tasks:** Streamline repetitive tasks like scheduling emails, managing calendars

### 4. 圖片生成文字

下載 圖片讀取套件

In [5]:
%pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [6]:
import google.generativeai as genai
import requests
from PIL import Image
from io import BytesIO

model = genai.GenerativeModel("gemini-1.5-flash")

url = 'https://preview.redd.it/r3cd5lf88yj81.png?auto=webp&s=2169f23199e99445ab2cee153771ee653b313603' # 這是一張圖片的網址
pic_response = requests.get(url)
pic = Image.open(BytesIO(pic_response.content)) # 大家不用理解這一行，只需要知道他是讀取圖片的方法


response = model.generate_content(["Tell me about this picture", pic],stream=True)
for i in response:
    print(i.text)

KeyboardInterrupt: 

### 5. 參數調整

In [ ]:
response = model.generate_content(
    'tell me something about the previous picture',
    generation_config = genai.GenerationConfig(max_output_tokens=1000,temperature=0.1,),
    stream=False
)
print(response.text)

- max_output_tokens: 生成文字的最大長度
- temperature: 創意度，0.1~ 2.0 ，數字越高創意度越高
- top_k: 決定取幾個機率最高的前幾個字。
- top_p: 取機率最高的前幾個字，數字越大創意度越高
- frequency_penalty: 決定重複字詞的機率
<br/>
......太多了，大家可以參考官方文件


### 6. Chat mode 聊天模式

前面的模式讓我們可以生成文字，但是不會產生聊天紀錄，每次的對話都是單次的<br/>
但是如果我們想要持續的跟AI對話呢？<br/>

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat()
response = chat.send_message("HI")
print(response.text)


msg = "Something to ask AI"
print("User:\n",msg)
response = chat.send_message(f'{msg}')
print("AI:")

print(response.text)


使用stream 方法來逐一地顯示輸出，減少時間斷裂感

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat()
response = chat.send_message("HI")
print(response.text)
try:
    while(True):
        msg = input("")
        print("User:\n",msg)
        response = chat.send_message(f'{msg}',stream=True)
        print("AI:")
        for chunck in response:
            print(chunck.text)
except:
    print ("Chat ended")

### 7. 程式碼執行模式

Gemini API 可以直接執行的程式碼，大家可以直接執行看看結果

In [7]:

model = genai.GenerativeModel(
    model_name='gemini-1.5-flash')

response = model.generate_content(('What is the sum of the first 50 prime numbers? '
    'Generate and run code for the calculation, and make sure you get all 50.'),
    tools='code_execution')

print(response.text)


To answer your question, I need to first generate the first 50 prime numbers and then calculate their sum.  I'll use a function to identify primes and then iterate to find the first 50.


``` python
def is_prime(n):
    """Checks if a number is prime."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

primes = []
num = 2
count = 0
while count < 50:
    if is_prime(num):
        primes.append(num)
        count += 1
    num += 1

print(f'{primes=}')
print(f'Sum of the first 50 prime numbers: {sum(primes)}')


```
```
primes=[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229]
Sum of the first 50 prime numbers: 5117


In [8]:
from IPython.display import display, Markdown
display(Markdown(response.text))

To answer your question, I need to first generate the first 50 prime numbers and then calculate their sum.  I'll use a function to identify primes and then iterate to find the first 50.


``` python
def is_prime(n):
    """Checks if a number is prime."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

primes = []
num = 2
count = 0
while count < 50:
    if is_prime(num):
        primes.append(num)
        count += 1
    num += 1

print(f'{primes=}')
print(f'Sum of the first 50 prime numbers: {sum(primes)}')


```
```
primes=[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229]
Sum of the first 50 prime numbers: 5117

```
The code generates the first 50 prime numbers and calculates their sum. The sum of the first 50 prime numbers is 5117.


當然，也可以將Code Execution 寫成對話形式

In [ ]:
from IPython.display import display, Markdown
model = genai.GenerativeModel(model_name='gemini-1.5-pro',
                              tools='code_execution')

chat = model.start_chat()

response = chat.send_message((
    'What is the sum of the first 50 prime numbers? '
    'Generate and run code for the calculation, and make sure you get all 50.'))


display(Markdown(response.text))